In [ ]:
!pip install pandas


In [ ]:
!pip install dython

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
import os, time

folder_path = r"C:\Users\aparn\Box\Data (Matthew Rizzo)\DataAnalysis\FormattingQAChecks\ActiGraph_FormattingQAChecks\ActiGraph_FormatQA_Data\Sleep\Data\Sleep_Formatted_Centrepoint\CleanSingleNightSleep_Formatted_Centrepoint"

print("exists:", os.path.exists(folder_path))
print("isdir:", os.path.isdir(folder_path))

# sometimes a short pause helps if Box is re-syncing
time.sleep(2)
print("exists after 2s:", os.path.exists(folder_path))


## Read files

In [ ]:
import os
import pandas as pd
import concurrent.futures

# Define the path to the folder
folder_path = r"C:\Users\aparn\Box\Data (Matthew Rizzo)\DataAnalysis\FormattingQAChecks\ActiGraph_FormattingQAChecks\ActiGraph_FormatQA_Data\Sleep\Data\Sleep_Formatted_Centrepoint\CleanSingleNightSleep_Formatted_Centrepoint"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Function to read each CSV file
def read_csv(file):
    file_path = os.path.join(folder_path, file)
    return pd.read_csv(file_path)

# Print number of CSV files to check if it's large
print(f"Number of CSV files: {len(csv_files)}")

# Use ThreadPoolExecutor to read the files in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Map the read_csv function to each file in csv_files
    dfs = list(executor.map(read_csv, csv_files))

# Concatenate all DataFrames into a single DataFrame
sleep_df = pd.concat(dfs, ignore_index=True)


In [ ]:
len(sleep_df)

In [ ]:
sleep_df.head(5)

# Count the imputed data


In [ ]:
# Count the number of True values in 'sleep_median' column
sleep_median_true_count = sleep_df[sleep_df['sleep_median'] == True].shape[0]

# Print the result
print(f"The count of sleep_median == True is: {sleep_median_true_count}")


## Remove datapoints where Sleep_wear==false


In [ ]:
# Count the number of 'False' values in 'sleep_wear' for each unique 'subj'
false_counts = sleep_df.groupby('subj')['sleep_wear'].apply(lambda x: (x == False).sum()).reset_index()

# Rename columns for clarity
false_counts.columns = ['subj', 'false_count']


In [ ]:
false_counts.head()

In [ ]:
# Calculate the sum of the 'false_count' column
total_false_count = false_counts['false_count'].sum()

# Display the result
print(total_false_count)


In [ ]:
# Remove rows where 'sleep_wear' is False
sleep_df_filtered = sleep_df[sleep_df['sleep_wear'] != False].reset_index(drop=True)


In [ ]:
len(sleep_df_filtered)

In [ ]:
sleep_df_filtered.head()

## calculate week and date

In [ ]:
# Ensure 'inbed_time_utc' is in datetime format
sleep_df_filtered['inbed_time_utc'] = pd.to_datetime(sleep_df_filtered['inbed_time_utc'], errors='coerce')

# Extract only the date part (ignore the time) and calculate the week number
sleep_df_filtered['inbed_date_utc'] = sleep_df_filtered['inbed_time_utc'].dt.date  # Keep only the date part

# Now calculate the week number based on the date
sleep_df_filtered['week_number'] = pd.to_datetime(sleep_df_filtered['inbed_date_utc']).dt.isocalendar().week

# Check the result
print(sleep_df_filtered[['inbed_date_utc', 'week_number']].head())


In [ ]:
# Drop the 'week_number_cst' column
sleep_df_filtered = sleep_df_filtered.drop(columns=['inbed_date_utc'])

In [ ]:
# Assuming your DataFrame is named 'df'
sleep_df_filtered.to_csv('sleep_variables.csv', index=False)


In [ ]:
sleep_df_filtered.dtypes

In [ ]:
sleep_df_filtered.head()

In [ ]:
sleep_df_filtered['week_number'].nunique()     ### 52.14 weeks in a year

In [ ]:
# Group by 'subj' and 'week_number' and count the occurrences (size of each group)
week_counts = sleep_df_filtered.groupby(['subj', 'week_number']).size().reset_index(name='count')

# Find the maximum count from the grouped data
max_count = week_counts['count'].max()

# Print the maximum count
print(f"The maximum count of entries for a unique week number and subject combination is {max_count}")


In [ ]:
# Drop the 'week_number_cst' column
#sleep_df_filtered = sleep_df_filtered.drop(columns=['week_number_cst'])

In [ ]:
# Count the number of NaN values in the 'inbed_time_utc' column
nan_count = sleep_df_filtered['inbed_time_utc'].isna().sum()

# Print the result
print(f"The number of NaN values in 'inbed_time_utc' column is: {nan_count}")


In [ ]:
# Drop rows where 'inbed_time_utc' is NaN
sleep_df_filtered = sleep_df_filtered.dropna(subset=['inbed_time_utc'])

In [ ]:
# Ensure 'inbed_time_utc' is in datetime format if it isn't already
sleep_df_filtered['inbed_time_utc'] = pd.to_datetime(sleep_df_filtered['inbed_time_utc'], errors='coerce')

# Fill NaN values with a default year (e.g., 0 or any year you prefer)
sleep_df_filtered['year'] = sleep_df_filtered['inbed_time_utc'].dt.year.fillna(0).astype(int)



In [ ]:
# Group by 'subj' and 'week_number', and get the number of unique 'year' values for each group
year_counts = sleep_df_filtered.groupby(['subj', 'week_number'])['year'].nunique().reset_index(name='unique_year_count')

# Filter for rows where the count of unique 'year' is greater than 1
multiple_years = year_counts[year_counts['unique_year_count'] > 1]

# Get the list of subj names for these pairs
subj_names = multiple_years[['subj', 'week_number']]

# Print the result
print("The subj and week_number pairs where more than one unique year exists:")
print(subj_names)

# Get the count of such pairs
count_multiple_years = subj_names.shape[0]

# Print the count
print(f"The count of pairs where subj and week_number have more than one unique year is: {count_multiple_years}")


## calculate the metrics of sleep based on each subj, week and year

In [ ]:
# Group by 'subj', 'week_number', and 'year' and calculate the mean for the required columns
mean_values = sleep_df_filtered.groupby(['subj', 'week_number', 'year']).agg(
    mean_efficiency=('efficiency', 'mean'),
    sd_efficiency=('efficiency', 'std'),
    mean_tst=('asleep_tst_min', 'mean'),
    sd_tst=('asleep_tst_min', 'std'),
).reset_index()

# Add 'inbed_time_utc' and 'inbed_time_cst' to the new DataFrame
# For 'inbed_time_utc', we will use the first entry of each group (or any other aggregation as needed)
mean_values['inbed_time_utc'] = sleep_df_filtered.groupby(['subj', 'week_number', 'year'])['inbed_time_utc'].first().reset_index(drop=True)
mean_values['inbed_time_cst'] = sleep_df_filtered.groupby(['subj', 'week_number', 'year'])['inbed_time_cst'].first().reset_index(drop=True)


In [ ]:
mean_values.head(5)

In [ ]:
# Assuming your DataFrame is named 'df'
mean_values.to_csv('mean_sleep_variables_v1.csv', index=False)


## load FINAL_MODEL dataset AND ADD SLEEP VARIABLES

In [ ]:
# Define the file path
file_path = r'C:/Users/aparn/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/weekly_stats_demo_cleaned_for_model.csv'

# Read the CSV file into a DataFrame
final_df_model = pd.read_csv(file_path)

In [ ]:
final_df_model.head()

In [ ]:
# Merge the 'mean_values' DataFrame with 'final_df_model' based on 'subj', 'week_number', and 'year'
merged_df = final_df_model.merge(mean_values[['subj', 'week_number', 'year', 'mean_efficiency', 'sd_efficiency', 'mean_tst', 'sd_tst']],
                                 on=['subj', 'week_number', 'year'],
                                 how='left')


In [ ]:
merged_df.head()

In [ ]:
# Check for rows where 'mean_efficiency' is NaN after the merge
missing_rows = merged_df[merged_df['mean_efficiency'].isna()]

print(f"Rows in 'final_df_model' that have no match in 'mean_values':")
print(missing_rows[['subj', 'week_number', 'year']])


In [ ]:
# Assuming your DataFrame is named 'df'
merged_df.to_csv('weekly_stats_demo_cleaned_for_model_v3.csv', index=False)
